In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/zaza/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
PATH = 'data/sale_prediction/'

In [5]:
!ls {PATH}

item_categories.csv  sales_train.csv	    shops.csv
items.csv	     sample_submission.csv  test.csv


In [17]:
!head {PATH}sales_train.csv

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
02.01.2013,0,59,22154,999.0,1.0
03.01.2013,0,25,2552,899.0,1.0
05.01.2013,0,25,2552,899.0,-1.0
06.01.2013,0,25,2554,1709.05,1.0
15.01.2013,0,25,2555,1099.0,1.0
10.01.2013,0,25,2564,349.0,1.0
02.01.2013,0,25,2565,549.0,1.0
04.01.2013,0,25,2572,239.0,1.0
11.01.2013,0,25,2572,299.0,1.0


In [10]:
df_raw = pd.read_csv(f'{PATH}sales_train.csv', low_memory=False, parse_dates=["date"])

In [11]:
df_raw

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-09-10,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [24]:
add_datepart(df_raw, 'date')

In [25]:
df_raw

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,59,22154,999.00,1.0,2013,2,5,1,4,32,False,True,False,False,False,False,1359676800
1,0,25,2552,899.00,1.0,2013,3,9,1,4,60,False,True,False,False,False,False,1362096000
2,0,25,2552,899.00,-1.0,2013,5,18,1,2,121,False,True,False,False,False,False,1367366400
3,0,25,2554,1709.05,1.0,2013,6,22,1,5,152,False,True,False,False,False,False,1370044800
4,0,25,2555,1099.00,1.0,2013,1,3,15,1,15,False,False,False,False,False,False,1358208000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0,2015,10,41,10,5,283,False,False,False,False,False,False,1444435200
2935845,33,25,7460,299.00,1.0,2015,9,37,10,3,253,False,False,False,False,False,False,1441843200
2935846,33,25,7459,349.00,1.0,2015,10,42,14,2,287,False,False,False,False,False,False,1444780800
2935847,33,25,7440,299.00,1.0,2015,10,43,22,3,295,False,False,False,False,False,False,1445472000


In [27]:
df_raw.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day',
       'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')